In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from hera_cal import io
import glob
from astropy.time import Time
from IPython.display import display, HTML

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
#JD = 2459140
#DPATH = '/lustre/aoc/projects/hera/H4C/postprocessing/'
#LABEL = 'all-bands-allbls-long-delay-clean-include-variable-ants'


#get data location
DPATH = os.environ['DATA_PATH']
print(f'DATA_PATH = "{DPATH}"')
JD = os.environ['JULIANDATE']
print(f'JULIANDATE = {JD}')
utc = Time(JD, format='jd').datetime
print(f'Date = {utc.month}-{utc.day}-{utc.year}')
LABEL = os.environ['LABEL']

In [ ]:
calfiles_fsmooth = sorted(glob.glob(f'{DPATH}/{JD}/zen.{JD}.*.sum.{LABEL}.freq_smoothed_abs_degen_time_smoothed.calfits'))
calsols = io.read_hera_calfits(calfiles_fsmooth, read_flags=True)


In [ ]:
autofiles = sorted(glob.glob(f'{DPATH}/{JD}/zen.*.sum.autos.uvh5'))
hd_autos = io.HERADataFastReader(autofiles)
data_autos, _, _ = hd_autos.read()

In [ ]:
data_ants = hd_autos.data_ants
times = hd_autos.times
freqs = hd_autos.freqs
lsts = hd_autos.lsts

In [ ]:
import scipy.interpolate as interp
lst_func = interp.interp1d(hd_autos.times, hd_autos.lsts * 12 / np.pi)

In [ ]:
import warnings


# I expect to see RuntimeWarnings in this block
#ants = [11, 58, 179]
ants = data_ants
franges = [(50, 85), (105, 135), (140, 160), (160, 170), (0, 1000)]

with warnings.catch_warnings():

    for pol in ['ee', 'nn']:
        fig, axarr = plt.subplots(len(ants), len(franges))
        fig.set_size_inches(6 * len(franges), 6 * len(ants))
        
        warnings.simplefilter("ignore", category=RuntimeWarning)

        for antnum, ant in enumerate(ants):

            
            blk = (ant, ant, 'nn')
            gk = (ant, 'Jnn')

            autos_uncalib = data_autos[blk] /  ~calsols['flags'][gk][::-1]
            autos_calib = data_autos[blk] /  ~calsols['flags'][gk] / np.abs(calsols['gains'][gk]) ** 2.

            for fnum, frange in enumerate(franges):
                plt.sca(axarr[antnum][fnum])
                cmin = np.argmin(np.abs(freqs - frange[0] * 1e6))
                cmax = np.argmin(np.abs(freqs - frange[1] * 1e6))
                
                labels = ['Uncalibrated Auto', 'Calibrated Auto']
                for dnum, dat in enumerate([autos_uncalib, autos_calib]):

                    dat = 10 ** (np.log10(dat[:, cmin: cmax]))
                    dat[~np.isfinite(dat)] = np.nan




                    ms = dat / np.nanmean(dat, axis=0)#np.subtract(dat, np.nanmean(dat,axis=0))
                    ms = np.nanmean(ms, axis=1)

                    plt.plot(times, ms, label=labels[dnum])
                
                plt.legend()
                if fnum == 0:
                    plt.ylabel( f'Ant {ant}-{pol} \n' + '$\\langle \\frac{V_{aa}}{\\langle V_{aa} \\rangle_{time}} \\rangle_{freq}$ \n',
                               fontsize=24)
                
                if antnum == len(ants) - 1:
                    plt.xlabel('LST [hr]', fontsize=24)
                
                plt.ylim(.6, 2)
                plt.yscale('log')
                plt.gca().set_xticks([t for t in plt.gca().get_xticks() if t >= times.min() and t <= times.max()])
                plt.gca().set_xticklabels(f'{lst_func(t):.2f}' for t in plt.gca().get_xticks())
                if antnum == 0:
                    plt.title(f'{frange[0]} - {frange[1]} MHz', fontsize=24)
                
                plt.grid(which='both')
                plt.gca().tick_params(labelsize=16)
    




